In [3]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import optuna

C:\Users\Samantha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load Data

In [7]:
age_gender_df = pd.read_csv("alter_geschlecht.csv")
interests_df = pd.read_csv("interesse.csv")
rest_df = pd.read_csv("rest.csv")

Check Each dataframe

In [180]:
age_gender_null_value_stats = age_gender_df.isnull().sum(axis=0)
age_gender_null_value_stats[age_gender_null_value_stats != 0]

Series([], dtype: int64)

In [181]:
age_gender_df.head()

,id,Geschlecht,Alter
0,170727,Male,42
1,41795,Male,24
2,369321,Female,30
3,500463,Female,32
4,327695,Male,34


In [182]:
age_gender_df.describe()

,id,Alter
count,508146.000000,508146.000000
mean,254073.500000,38.808413
std,146689.259281,15.500179
min,1.000000,20.000000
25%,127037.250000,25.000000
50%,254073.500000,36.000000
75%,381109.750000,49.000000
max,508146.000000,85.000000


In [183]:
interests_null_value_stats = interests_df.isnull().sum(axis=0)
interests_null_value_stats[interests_null_value_stats != 0]

Series([], dtype: int64)

In [184]:
interests_df.head()

,id,Interesse
0,1,1.0
1,2,0.0
2,3,1.0
3,4,0.0
4,5,0.0


In [185]:
interests_df.describe()

,id,Interesse
count,381109.000000,381109.000000
mean,190555.000000,0.122563
std,110016.836208,0.327936
min,1.000000,0.000000
25%,95278.000000,0.000000
50%,190555.000000,0.000000
75%,285832.000000,0.000000
max,381109.000000,1.000000


In [186]:
rest_null_value_stats = rest_df.isnull().sum(axis=0)
rest_null_value_stats[rest_null_value_stats != 0]

Series([], dtype: int64)

In [187]:
rest_df.head()

,Fahrerlaubnis;Regional_Code;Vorversicherung;Alter_Fzg;Vorschaden;Jahresbeitrag;Vertriebskanal;Kundentreue;id
0,1;15.0;1;1-2 Year;No;2630.0;124.0;74;317635
1,1;28.0;0;1-2 Year;Yes;2630.0;125.0;213;337993
2,1;33.0;0;1-2 Year;Yes;27204.0;124.0;114;160325
3,1;46.0;1;< 1 Year;No;31999.0;152.0;251;141620
4,1;49.0;0;1-2 Year;Yes;28262.0;26.0;60;75060


In [188]:
rest_df.describe()

,Fahrerlaubnis;Regional_Code;Vorversicherung;Alter_Fzg;Vorschaden;Jahresbeitrag;Vertriebskanal;Kundentreue;id
count,508146
unique,508146
top,1;3.0;0;< 1 Year;Yes;54331.0;152.0;250;378827
freq,1


Properly separate features by column

In [189]:
rest_df[["Fahrerlaubnis", "Regional_Code", "Vorversicherung", "Alter_Fzg", "Vorschaden", "Jahresbeitrag", "Vertriebskanal","Kundentreue","id"]]= rest_df['Fahrerlaubnis;Regional_Code;Vorversicherung;Alter_Fzg;Vorschaden;Jahresbeitrag;Vertriebskanal;Kundentreue;id'].str.split(';', n=-1, expand=True)

In [190]:
rest_df = rest_df.drop("Fahrerlaubnis;Regional_Code;Vorversicherung;Alter_Fzg;Vorschaden;Jahresbeitrag;Vertriebskanal;Kundentreue;id", axis=1)
rest_df.head()

,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue,id
0,1,15.0,1,1-2 Year,No,2630.0,124.0,74,317635
1,1,28.0,0,1-2 Year,Yes,2630.0,125.0,213,337993
2,1,33.0,0,1-2 Year,Yes,27204.0,124.0,114,160325
3,1,46.0,1,< 1 Year,No,31999.0,152.0,251,141620
4,1,49.0,0,1-2 Year,Yes,28262.0,26.0,60,75060


In [191]:
rest_df.describe()

,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue,id
count,508146,508146,508146,508146,508146,508146,508146,508146,508146
unique,2,53,2,3,2,52062,157,290,508146
top,1,28.0,0,1-2 Year,Yes,2630.0,152.0,256,378827
freq,507097,141937,275076,267015,256248,86488,179523,1877,1


In [192]:
rest_df['id']= rest_df['id'].astype(int)

Merge data

In [193]:
merged_df = pd.merge(age_gender_df, rest_df, on='id', how='outer')
merged_df.head()

,id,Geschlecht,Alter,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39


In [194]:
merged_df =pd.merge(merged_df, interests_df, on='id', how='outer')

Clean feature formatting (numerical and categorical features)

In [195]:
merged_df['Regional_Code']= merged_df['Regional_Code'].astype(float).astype(int)
merged_df['Vertriebskanal']= merged_df['Vertriebskanal'].astype(float).astype(int)
merged_df.head()

,id,Geschlecht,Alter,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue,Interesse
0,1,Male,44,1,28,0,> 2 Years,Yes,40454.0,26,217,1.0
1,2,Male,76,1,3,0,1-2 Year,No,33536.0,26,183,0.0
2,3,Male,47,1,28,0,> 2 Years,Yes,38294.0,26,27,1.0
3,4,Male,21,1,11,1,< 1 Year,No,28619.0,152,203,0.0
4,5,Female,29,1,41,1,< 1 Year,No,27496.0,152,39,0.0


In [196]:
merged_df_null_value_stats = merged_df.isnull().sum(axis=0)
merged_df_null_value_stats[merged_df_null_value_stats != 0]

Interesse    127037
dtype: int64

In [197]:
unknown_rows= merged_df[merged_df['Interesse'].isna()]
unknown_rows.head()

,id,Geschlecht,Alter,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue,Interesse
381109,381110,Male,25,1,11,1,< 1 Year,No,35786.0,152,53,NaN
381110,381111,Male,40,1,28,0,1-2 Year,Yes,33762.0,7,111,NaN
381111,381112,Male,47,1,28,0,1-2 Year,Yes,40050.0,124,199,NaN
381112,381113,Male,24,1,27,1,< 1 Year,Yes,37356.0,152,187,NaN
381113,381114,Male,27,1,28,1,< 1 Year,No,59097.0,152,297,NaN


Remove rows where interest of customer is unknown

In [198]:
merged_df = merged_df[merged_df['Interesse'].notna()]
merged_df_null_value_stats = merged_df.isnull().sum(axis=0)
print(merged_df_null_value_stats[merged_df_null_value_stats != 0])


Series([], dtype: int64)


In [199]:

merged_df['Interesse'] = merged_df['Interesse'].astype(bool)
merged_df.head()

,id,Geschlecht,Alter,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue,Interesse
0,1,Male,44,1,28,0,> 2 Years,Yes,40454.0,26,217,True
1,2,Male,76,1,3,0,1-2 Year,No,33536.0,26,183,False
2,3,Male,47,1,28,0,> 2 Years,Yes,38294.0,26,27,True
3,4,Male,21,1,11,1,< 1 Year,No,28619.0,152,203,False
4,5,Female,29,1,41,1,< 1 Year,No,27496.0,152,39,False


In [200]:
merged_df['Alter_Fzg'].unique()

array(['> 2 Years', '1-2 Year', '< 1 Year'], dtype=object)

In [201]:
mappings = {
    '> 2 Years': '2',
    '1-2 Year': '1',
    '<1 Year': '0'
}  
merged_df['Alter_Fzg']= merged_df['Alter_Fzg'].map(mappings)

In [202]:
merged_df['Alter_Fzg']= merged_df['Alter_Fzg'].fillna('0')
merged_df['Alter_Fzg']= merged_df['Alter_Fzg'].astype(int)

In [203]:
merged_df['Alter_Fzg'].unique()

array([2, 1, 0])

In [205]:
merged_df['Kundentreue']= merged_df['Kundentreue'].astype(int)
merged_df['Jahresbeitrag']= merged_df['Jahresbeitrag'].astype(float)
merged_df['Fahrerlaubnis']= merged_df['Fahrerlaubnis'].astype(bool)
merged_df['Vorversicherung']= merged_df['Vorversicherung'].astype(bool)
merged_df['Vorschaden']= merged_df['Vorschaden'].astype(bool)

In [206]:
merged_df.dtypes

id                   int64
Geschlecht          object
Alter                int64
Fahrerlaubnis         bool
Regional_Code        int64
Vorversicherung       bool
Alter_Fzg            int64
Vorschaden            bool
Jahresbeitrag      float64
Vertriebskanal       int64
Kundentreue          int64
Interesse             bool
dtype: object

In [207]:
merged_df.head()

,id,Geschlecht,Alter,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue,Interesse
0,1,Male,44,True,28,True,2,True,40454.0,26,217,True
1,2,Male,76,True,3,True,1,True,33536.0,26,183,False
2,3,Male,47,True,28,True,2,True,38294.0,26,27,True
3,4,Male,21,True,11,True,0,True,28619.0,152,203,False
4,5,Female,29,True,41,True,0,True,27496.0,152,39,False
